In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
## Custom functions
import FockSystem.FockSystem as fst

# Constructing operator sequences

## Defining sequences

<span style="font-size:20px; font-weight:bold;">From Ints and Lists</span>


In [ ]:
fst.OperSequence(0)

In [ ]:
fst.OperSequence(1)

In [ ]:
fst.OperSequence(1,2)

In [ ]:
fst.OperSequence([0,1],[1,2])

<span style="font-size:20px; font-weight:bold;">From Tuples</span>


In [ ]:
fst.OperSequence((('c',0,'up'),('c',1,'down')), ('c',0,'down'), ('a',2,'up'))

<span style="font-size:20px; font-weight:bold;">From Strings</span>


In [ ]:
## Following OpenFermions convention, strings will be parsed in a (somewhat) similar manner
## Mismatch for now is that 'site 1' is displayed automatically as site 0 spin-up, rather than freely allowing for interpretation of 'site 1'

In [ ]:
fst.OperSequence('0^,1', '1^,0,2^')

## Assigning weights

In [ ]:
## The weights of operator sequences can be set in many ways

In [ ]:
c_down = fst.OperSequence(0, weights = [1j])
c_up = fst.OperSequence(2, weights = [5])
op = c_down + c_up
op

In [ ]:
## A OperSequence instance can be used to set the weight of that subsequence in a longer sequence

In [ ]:
op = c_down + c_up
op[c_up] = 10
op[c_down] = 20
op

<span style="font-size:20px; font-weight:bold;">Shorthand codes</span>


In [ ]:
## The 'greater than' operator has additionally been reserved to quickly assign values

In [ ]:
## Passing a single value assigns the same value to each subsequence
op > 3
op

In [ ]:
## Passing an array assigns the values in order
op > [5,2]

# Supported Operator methods

## Basic operations

<span style="font-size:20px; font-weight:bold;">Addition and Subtraction</span>


In [ ]:
c_down = fst.OperSequence(0)
c_up = fst.OperSequence(2)

In [ ]:
c_down*(c_down >>1)

In [ ]:
c_up - c_down + (~c_up) + 10

<span style="font-size:20px; font-weight:bold;">Multiplication and Division</span>


In [ ]:
4*c_down*c_up

In [ ]:
## Normal ordering is automatically applied for multiplication
4*c_up*c_down

In [ ]:
test = c_up*c_down
test

In [ ]:
test

In [ ]:
test

In [ ]:
H = c_up*(~c_up) + c_up*(~c_up>>1)
H

In [ ]:
N = 7
H = H >> np.arange(1,N)
H

In [ ]:
test >> np.array([1,2,5,6])

<span style="font-size:20px; font-weight:bold;">Exponentation</span>


In [ ]:
## Creating an operator t from some paper that should give t**4 = -1
an, cr, up, dwn = "a", "c", "u", "d"
weights = [1j, -1, 1, -1, 1j, -1j]
operators = [
    ((an, 0, up),(cr, 0, up), (an, 0, dwn)),
    ((an, 0, dwn), (cr, 0, dwn),(cr, 0, up)),
    (an, 0, up),
    ((an, 0, dwn),(cr, 0, dwn), (an, 0, up)),
    (cr, 0, dwn),
    ((an, 0, up),(cr, 0, up), (cr, 0, dwn)),
]
t = fst.OperSequence(*operators, weights=weights)

In [ ]:
t 

In [ ]:
t**2

In [ ]:
t**3

In [ ]:
t**4

## Additional Operations

<span style="font-size:20px; font-weight:bold;">Site shifting</span>


In [ ]:
## The lshift and rshift operators will shift the entire sequence by the specified numbers of site

In [ ]:
c_down_0 = fst.OperSequence(0, 8)
c_down_0 >> 6

In [ ]:
c_down_5 = fst.OperSequence(('c',5,'dwn'), ('c',9,'up'))
c_down_5 << 4

In [ ]:
H_base = c_up*(~c_up) + c_up*(~c_up>>1)
H_base

In [ ]:
## Shift over an array of numbers
N = 3
H = H_base >> np.arange(1,N)
H

<span style="font-size:20px; font-weight:bold;">Conjugation</span>


In [ ]:
arbitrary_sequence = fst.OperSequence([0,1],[4,12],[3,11],[5,6,7])
arbitrary_sequence

In [ ]:
## Arbitraty sequence conjugated:
~arbitrary_sequence

<span style="font-size:20px; font-weight:bold;">Normal Ordering</span>


In [ ]:
## Create example
example_seq = fst.OperSequence([1,0],[5,6,7])
example_seq

In [ ]:
## Normal order the example
example_seq.normal_order()
example_seq

# Connecting to Fock states

In [ ]:
## The Fock State Class is interpreted by the OperSequence class to generate the data that represents a Hamiltonian

In [ ]:
## Create by passing number of Fermionic sites
basis = fst.FockStates(3)
basis

In [ ]:
## Create by passing array of ints that represent specific states
basis = fst.FockStates([0,1,2,5,8])
basis

<span style="font-size:20px; font-weight:bold;">Restricting the Fock Space</span>


In [ ]:
basis = fst.FockStates(3)
even_states = basis.restrict(parity='even')
even_states

In [ ]:
basis = fst.FockStates(3)
only_spin_down = basis.restrict(Ez_inf=True,U_inf=True)
only_spin_down

<span style="font-size:20px; font-weight:bold;">Calculating Hamiltonian action on a basis</span>


In [ ]:
## Define basic elements for convenience
c_down = fst.OperSequence(0)
c_up = fst.OperSequence(2)
a_up = ~c_up
a_down = ~c_down

## Construct basic Hamiltonian
H = c_down*a_down + 20*c_down*(c_down >>1) + 20*c_down*(a_down>>1)
H

In [ ]:
## Define basis
basis = fst.FockStates(3)
only_spin_down_basis = basis.restrict(Ez_inf=True,U_inf=True)
only_spin_down_basis

In [ ]:
H_data = H[only_spin_down_basis]
H_as_array = H_data.to_array()
H_sparse = H_data.to_sparse_coo()
H_as_array

<span style="font-size:20px; font-weight:bold;">Block diagonalization</span>


In [ ]:
## The function .to_block_diagonal_basis() is provided to return a new FockStates instance that has been ordered to be block diagonal in the coupled OperSequence

In [ ]:
bd_basis = H[only_spin_down_basis].to_block_diagonal_basis()
fig,axs = plt.subplots(ncols = 2)
axs[0].set_title("Original basis")
axs[0].matshow(np.real(H[only_spin_down].to_array()))
axs[1].set_title("Block diagonal ordered basis")
axs[1].matshow(np.real(H[bd_basis].to_array()))

<span style="font-size:20px; font-weight:bold;">Matrix-Vector Multiplication</span>


In [ ]:
random_vector = np.array([np.random.randint(10) for _ in  range(len(only_spin_down_basis.states))], dtype=complex)

## The @ symbol implements matrix-vecotr multipliction of OperSequence in a selected basis
print("Direct matrix-vector product")
result = H[only_spin_down] @ random_vector
print(np.array(result))

## This is identical to converting first to an array and handling the matrix-vector product there
## The direct method becomes faster and less memory-intensive for larger systems (N > 5)
array = H[only_spin_down_basis].to_array()
print("Normal matrix-vector product")
result = array @ random_vector
print(result)

# Minimal Example - The effective kitaev chain


<span style="font-size:20px; font-weight:bold;">Constructing the Hamiltonian from scratch</span>


In [ ]:
## Define basic elements for convenience
c_down = fst.OperSequence(0)
c_up = fst.OperSequence(2)
a_up = ~c_up
a_down = ~c_down

In [ ]:
## Build the Hamiltonian
## By building from subsequences, the value of individual terms can be easily accessed later
N = 2

ECT = c_down*(~c_down>>1)
ECT = ECT >> np.arange(1,N-1)
    
CAR = c_down*(c_down>>1)
CAR = CAR >> np.arange(1,N-1)

MU = c_down*(~c_down)
MU = MU >> np.arange(1,N)

H = MU + CAR +ECT
H

In [ ]:
## Define a basis
basis = fst.FockStates(N)
basis = basis.restrict(Ez_inf=True,U_inf=True)

In [ ]:
ECT

In [ ]:
H[ECT] = 5

In [ ]:
H

In [ ]:
H[ECT] = 5
H[CAR] = 10
H[MU[0]] = 6
H[MU[1]] = 7

In [ ]:
H[ECT] = 12

In [ ]:
H[basis]

In [ ]:
H[basis].to_array()

In [ ]:
H[basis].to_sparse_coo()

In [ ]:
## Optional: Split basis into odd and even and restrict to inf U and inf Ez (i.e., exclude double occupation and spin-up occupation states)
even_basis = basis.restrict('even',Ez_inf=True,U_inf=True)
even_array = H[even_basis].to_array()
odd_basis = basis.restrict('odd',Ez_inf=True,U_inf=True)
odd_array = H[odd_basis].to_array()

<span style="font-size:20px; font-weight:bold;">Phase diagrams</span>


<span style="font-size:20px; font-weight:bold;">Pre-defined functions</span>


In [ ]:
import Analysis.transport_tools as tu
from Analysis.systems import kitaev_chain, kramers_chain

In [ ]:
N=3
MU,CAR,ECT = kitaev_chain(N)
H = MU + CAR +ECT
basis = fst.FockStates(N)
even_basis = basis.restrict('even',Ez_inf=True,U_inf=True)
odd_basis = basis.restrict('odd',Ez_inf=True,U_inf=True)

H[CAR] = 50
H[ECT] = 50

In [ ]:
H[basis]

In [ ]:
mu_range = np.linspace(-100,100,100)
t_range = np.linspace(0,80,100)

## Loop over range and get array, pass to linalg for eigenvalues
result = tu.phase_diagram(H,odd_basis,even_basis, ECT, t_range, MU, mu_range)

fig,ax = plt.subplots(ncols=1, figsize = (3,3))
result['E'].plot()

<span style="font-size:20px; font-weight:bold;">Lead Transitions</span>


In [ ]:
N=3
MU,CAR,ECT = kitaev_chain(N)
H = MU + CAR +ECT

In [ ]:
H[CAR] = 20
H[CAR[1]]=-20
H[ECT] = 20

In [ ]:
basis = fst.FockStates(N)
inf_Ez_basis = basis.restrict(Ez_inf=True,U_inf=True)
array = H[inf_Ez_basis].to_array()

In [ ]:
fig, axs = plt.subplots(ncols = N, figsize = (3*N,3))
for ax in axs:
    ax.set_ylim([-100,100])
    
tu.energy_spectrum(H, inf_Ez_basis, MU[1], np.linspace(-100,100,100), np.arange(N), fig,axs)

plt.tight_layout()

<span style="font-size:20px; font-weight:bold;">Conductance</span>


In [ ]:
N = 3
MU,CAR,ECT = kitaev_chain(N)
H = MU + CAR + ECT

H[MU] = 0

In [ ]:
basis = fst.FockStates(N)
inf_Ez_basis = basis.restrict(Ez_inf=True,U_inf=True)
array = H[inf_Ez_basis].to_array()

In [ ]:
H[CAR] = 20e-3
H[ECT] = 20e-3

In [ ]:
lead_params = {"gammas": [0.001]*5, "kBT": 0.002, "dV": 0.001}
bias_range = np.linspace(-70e-3,70e-3, 100)
mu_range = np.linspace(-50e-3,50e-3,100)

In [ ]:
Gs = tu.conductance_spectrum(H,inf_Ez_basis,MU[0], mu_range, bias_range,sites = np.arange(N), lead_params = lead_params)

fig, axs = plt.subplots(ncols = N,figsize = (N*3.5,2.5))
for i in range(len(axs)):
    Gs[f'G_{i}{i}'].plot(ax =axs[i], cmap='magma')
plt.tight_layout()